# Load Data

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 

In [10]:
X_train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [40]:
X = torch.tensor(X_train.iloc[:, 1:].values).float()

In [41]:
X.shape # (example, image size)

torch.Size([42000, 784])

In [42]:
y = torch.tensor(X_train.iloc[:, :1].values)

In [43]:
y.shape

torch.Size([42000, 1])

In [44]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(784, 200)
        self.l2 = nn.Linear(200, 10)
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(F.relu(x))
        return x

In [64]:
myNet = Net()
optimizer = torch.optim.Adam(myNet.parameters(), lr=0.01)
epochs = 1000

for i in range(epochs+1):
    optimizer.zero_grad()
    ix = torch.randint(0, X.shape[0], (64,))
    logits = myNet(X[ix])
    loss = F.cross_entropy(logits, y[ix].squeeze())
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(f"Epoch {i} / {epochs} Loss: {loss.item()}")

Epoch 0 / 1000 Loss: 49.406105041503906
Epoch 100 / 1000 Loss: 1.1855688095092773
Epoch 200 / 1000 Loss: 0.5101887583732605
Epoch 300 / 1000 Loss: 0.6909492015838623
Epoch 400 / 1000 Loss: 0.7854114770889282
Epoch 500 / 1000 Loss: 0.2872884273529053
Epoch 600 / 1000 Loss: 0.5885886549949646
Epoch 700 / 1000 Loss: 0.9194672703742981
Epoch 800 / 1000 Loss: 0.6654410362243652
Epoch 900 / 1000 Loss: 0.4323260486125946
Epoch 1000 / 1000 Loss: 0.6369647979736328


In [81]:
raw_outs = myNet(torch.tensor(X_test.values).float())
probs = F.softmax(raw_outs, dim=1)
outs = torch.argmax(probs, dim=1)
outs.shape

torch.Size([28000])

In [ ]:
data = {
    "ImageId": torch.arange(1, 28001),  # List of Image IDs
    "Label": outs                       # Corresponding labels
}
df = pd.DataFrame(data)

In [79]:
df.to_csv("output.csv", index=False)